In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                  # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../../../")      #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
#using MetaGraphs
#using MetaGraphsNext
using GraphPlot
using SimpleWeightedGraphs
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff
using DelimitedFiles
using JSON
using Dates
using DataStructures
using Compose
using Compose: cm
using StatsBase

# Utils

## Enumerator

Tools to compute a running enumeration of a "list" of objects.

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

## Running Statistics

Tools to compute running statistics such as mean and std from a stream of numbers.

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

## Disjoint Union Set

* [https://en.wikipedia.org/wiki/Disjoint-set_data_structure](https://en.wikipedia.org/wiki/Disjoint-set_data_structure)

In [ ]:
mutable struct DUS
    parent::Vector{Int}
    rank::Vector{Int}
    nds::Int # number of disjoint sets
end

function DUS(n::Int64)
    return DUS(collect(1:n),zeros(Int64,n),n)
end

function find(dus::DUS,i::Int64)
    if dus.parent[i] != i
        dus.parent[i] = find(dus,dus.parent[i])
    end
    return dus.parent[i]
end

function unite(dus::DUS,i::Int64,j::Int64)
    root_i = find(dus,i)
    root_j = find(dus,j)
    if root_i==root_j
        return
    end
    if dus.rank[i] < dus.rank[j]
        dus.parent[root_i] = root_j
    elseif dus.rank[i] > dus.rank[j]
        dus.parent[root_j] = root_i
    else
        dus.parent[root_j] = root_i
        dus.rank[root_i] += 1
    end
    dus.nds -= 1
    @assert dus.nds > 0    
end

function together(dus::DUS,i::Int64,j::Int64)
    return find(dus,i) == find(dus,j)
end

function Base.print(dus::DUS)
    println("DUS.nds = ",dus.nds)
    for i in 1:length(dus.rank)
        println(i," => ",find(dus,i))
    end
end

### Test

## Coefficient of determiantion, $R^2$

Tools to compute the $R^2$ coefficient to evaluate the fit of functions.

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

## Log-range

Tools to create a range of numbers but in logarithmic scale.

In [ ]:
function logrange(start,stepmul,stop)
    @assert start>0
    @assert stepmul>1
    r = []
    s = start
    while s <= stop
        push!(r,s)
        s *= stepmul
    end
    return r
end

### Test

## Network stuff

In [ ]:
function main_component(g)
    c = connected_components(g)
    _,i = findmax(length.(c))
    return g[c[i]]
end

# Simplicial Complex

Tools to represent simplicial complexes and associated operators in matrix form. For further details, please read de manuscript.

The array `K[k]` is a representation of $K_k$ and has dimensions $m_k\times(k+1)$ and is used to enumerate the $k$-simplices of the complex.
Here $m_k=|K_k|=\dim C_k=\dim C_k^*$ is the number of $k$-simplices in the simplicial complex.
Especifically, `K[k][l,:]` is an array `[i_0,i_1,...,i_k]` denoting the $l$-th $k$-simplex of the complex.
By assumption, $i_0<i_1<...<i_k$.
The dictionary `I[k]` is the inverse mapping of `K[k]`.
It maps arrays `[i_0,i_1,...,i_k]` to their corresponding index `l` in the enumeration.

## Matricial representation of the (co)-boundary and (co)-differential operators

Recall that, if

$$
\partial_k(e_s) = (\alpha_k)_s{}^r e_r
$$
$$
\partial_k^*(e_s) = (\mu_k)_s{}^r e_r 
$$
$$
d_k(e^s) = (\beta_k)^s{}_r e^r
$$
$$
d_k^*(e^s) = (\mu_k)^s{}_r e^r
$$

and $m_{sr}=\delta_{sr}$, then

$$
(\beta_k)^s{}_r = (\alpha_{k+1})_r{}^s
$$
$$
(\mu_k)_s{}^r = (\alpha_{k})_r{}^s
$$
$$
(\nu_k)^s{}_r = (\beta_{k})^r{}_s = (\alpha_{k+1})_s{}^r
$$

and, therefore

$$
(\partial_k(c))^r = (\alpha_k^T)^r{}_sc^s
$$
$$
(\partial_k^*(c))^r = (\alpha_k)^r{}_sc^s
$$
$$
(d_k(f))_r = (\alpha_{k+1})_r{}^sf_s
$$
$$
(d_k^*(f))_r = (\alpha_{k+1}^T)_r{}^sf_s
$$

Finally, note that $\alpha_k\in \mathbb{R}^{m_{k}\times m_{k-1}}$ for $k\in 1,2,...,\kappa$ and

$$
(\alpha_{k})_{i_0...i_j...i_k}{}^{i_0...\hat{i}_j...i_k} = (-1)^j
$$

Here, $d_0$ and $d_1$ are needed. 
Hence, the matrices $\beta_0=\alpha_1$ and $\beta_1=\alpha_2$ have to be computed, respectively. 
In particular, assuming $i<j<k$, it is found that

$$(\alpha_1)_{ij}{}^{j} = 1$$
$$(\alpha_1)_{ij}{}^{i} = -1$$

and

$$(\alpha_2)_{ijk}{}^{jk} = 1$$
$$(\alpha_2)_{ijk}{}^{ik} = -1$$
$$(\alpha_2)_{ijk}{}^{ij} = 1$$

In [ ]:
struct SimplicialComplex
    K::Any
    I::Any
    alpha::Any
end

In [ ]:
"""
Number of dimensions (kappa+1) of a simplicial complex, k = 0,1,...,kappa.
"""
function ndim(sc::SimplicialComplex)
    return length(sc.K)
end

In [ ]:
"""
Maximum dimension of a simplicial complex.
"""
function kappa(sc::SimplicialComplex)
    return ndim(sc)-1
end

In [ ]:
"""
m_k = dim C_k = dim C_k^*.
"""
function m_k(sc::SimplicialComplex,k::Int)
    if k>=0 && k<=kappa(sc)
        return size(sc.K[k])[1]
    end
    return 0
end

In [ ]:
"""
List of the numbers of non-zero elements in the matricial representations of the (dual)-(co)-boundary operators, alpha_k.
"""
function nnz_alpha(sc::SimplicialComplex)
    return [nnz(sc.alpha[k]) for k in 1:length(sc.alpha)] 
end

In [ ]:
"""
Takes as input a list I of dictionaries I_k mapping simplices [i_0,i_1,...,i_k] to their corresponding enumeration numbers l.
Returns a SimplicialComplex containing:
    K : the list of the inverse mappings K_k of the I_k, and
    I : the list of the mappings I_k passed as input,
    alpha : the list alpha of the corresponding matrices alpha_k.
"""
function I_to_SC(I)
    # Create K
    K = []
    for k in 0:(length(I)-1)
        Kk = zeros(Int64,(length(I[k]),k+1))
        for (s,l) in I[k]
            Kk[l,:] = s
        end
        push!(K,Kk)
    end
    K = OffsetArray(K,0:(length(I)-1))

    # Create A
    alpha = []
    for k in 1:(length(I)-1)
        #@show k
        Kk=K[k]
        Kp=K[k-1] # p = k-1
        Ik=I[k]
        Ip=I[k-1]
        n_k = size(Kk)[1]
        n_p = size(Kp)[1]        
        rows = zeros(Int64,(k+1)*n_k)
        cols = zeros(Int64,(k+1)*n_k)
        vals = zeros(Float64,(k+1)*n_k)
        t = 0 # To enumerate the entries of the sparse matrix.
        for l in 1:n_k
            s = Kk[l,:]
            for j in 1:(k+1)
                r = vcat(s[1:(j-1)],s[(j+1):end])
                m = Ip[r]
                t+=1
                rows[t] = l
                cols[t] = m
                vals[t] = 1-2*((j-1)%2)
                #@show s,l,r,m,t,vals[t]
            end
        end
        alpha_k = sparse(rows,cols,vals,n_k,n_p)
        push!(alpha,alpha_k)
    end
    return SimplicialComplex(K,I,alpha)
end

In [ ]:
"""
Takes as input a graph g and returns the list of I_k for k=0,1,2.
"""
function g_to_3_cliques(g)
    # Generate I_0
    I0 = Dict([i] => i for i in 1:nv(g))    
    
    # Generate I_1
    I1 = Dict()    
    for (l,e) in enumerate(edges(g))
        i = src(e)
        j = dst(e)
        I1[[i,j]] = l
    end
    
    # Generate I_2
    I2 = Dict()
    t = 0
    for i in 1:nv(g)
        for j in neighbors(g,i)
            if i<j
                for k in neighbors(g,j)
                    if j<k && has_edge(g,k,i)
                        t += 1
                        I2[[i,j,k]] = t
                    end
                end
            end
        end
    end
    
    I = OffsetArray([I0,I1,I2],0:2)
    return I
end

### Tests

#### Testing `g_to_3_cliques` and `I_to_SC` 

# Hodge decomposition of $f$

Tools to compute the Hodge decomposition of a 1-chain $f$.

Recall that any $f\in C_1^*$ admits an unique decomposition $f=s+h+g$, where there exists $w\in C_0^*$ such that 

$$
L_0(w)=L^{\uparrow}_0(w)=d_0^*(d_0(w))=d_0^*(g)=d_0^*(f)
$$

and there exists $u\in C_2^*$ such that

$$
L_2^{\downarrow}(u)=d_1(d_1^*(u))=d_1(s)=d_1(f)
$$

In this way, we can find $w$ by solving $L^{\uparrow}_0(w)=d_0^*(f)$ and $u$ by solving $L_2^{\downarrow}(u)=d_1(f)$. Then, we can find $g=d_0(w)$, $s=d_1^*(u)$ and $h=f-g-s$.

## Matricial form

The component-wise form of equation $d_0^*(d_0(w))=d_0^*(f)$ is

$$
(\alpha_1^T)_r{}^s(\alpha_1)_s{}^qw_q = (\alpha_1^T)_r{}^sf_s
$$

or, matricially

$$
\alpha_1^T \alpha_1 w = \alpha_1^T f \;\;\;\; (1)
$$

Similarly, the component-wise form of equation $g = d_0(w)$ is

$$
(\alpha_1)_r{}^s w_s
$$

or, matricially

$$
g = \alpha_1 w
$$

Similarly, the matricial form of equation $d_1(d_1^*(u))=d_1(f)$ is

$$
\alpha_2 \alpha_2^T u = \alpha_2 f \;\;\;\; (2)
$$

and, the matricial form of equation $s=d_1^*(u)$ is

$$
s = \alpha_2^T u
$$

### Test example of Eqs. (18) and (20) in [2].

Eq. (18) of [2] is the analogous of Eq. (1) where $w$ in Eq. (1) is $\alpha$ in Eq. (18) of [2] and $f$ in Eq. (1) is the analogous of $w$ in Eq. (18) of [2].
The matrix of Eq. (18) in [2] is analogous to our $L_0=\alpha_1^T\alpha_1$.
Similarly, Eq. (20) of [2] is the analogous of Eq. (2) where $u$ in Eq. (2) is the analogous of $\gamma$ in Eq. (20) of [2] and $f$ in Eq. (2) is the analogous of $w$ in Eq. (20) of [2].
The matrix of Eq. (20) in [2] is analogous to our $L_2^{\downarrow}=\alpha_2\alpha_2^T$.

The $k$-clique complex of $g$ is the simplicial complex obtained by considering all the $q$-cliques of $g$ for $q\leq k$.

First, the 3-clique complex of the network of Fig. (1) in [2] is created.

Here, $z:=d_0^*(f)$ should equal the r.h.s. of Eq. (18) in [2].

Here, $L_0$ should equal the matrix at the l.h.s. of Eq. (18) in [2].

Next vector should be equal to the vector $\alpha$ reported in Table 5 of [2].

Next vector should equal the r.h.s. of Eq. (18) in [2].

Check that vector $g=d_0(w)=\alpha_1 w$ equals the vector $w_g$ of Table 4 in [2].

Check equality up to numerical tolerance that $d_1(f)=d_1(g)$.

Check the form of $L_2^{\downarrow}=\alpha_2\alpha_2^T$. It should equal the matrix at the l.h.s. of Eq. (20) in [2].

Here, $y:=d_1(f)$ should equal the r.h.s. of Eq. (20) in [2].

The vector $u$ obtained is different from the vector $\gamma$ reported in Table (5) of [2], which is the following one:

Is the vector $u_{\mathrm{johnson}}$ (i.e. $\gamma$) reported in [2] also a solution of Eq. (20) of [2]? We check:

It actually is! What about our solution? Let us check:

It is as well! Hence, there are multiple solutions, so $L_2^{\downarrow}(u)=y$ is a degenerate (i.e. underdetermined) linear system. 

Only one of the solutions of degenerate linear systems of a positive semidefinite linear operator has minimum norm $|x|$. Therefore, we ask ourselves which of the solutions to $L_2^{\downarrow}u=y$ we have found, has the smallest norm? Let us see:

So our solution has smaller norm than that of [2]. Hence, it is a better candidate for the unique solution of minimum norm.

Next, compute $s=d_1^*(u)=\alpha_2^T u$.

Check the "equality" of $\alpha_2 s = d_1(s) = d_1(f) = \alpha_2 f$.

Compute $h$.

Finally check that $L_1(h)=0$.

## Compute Hodge decomposition

The linear equations that should be solved are numerically challenging. Hence, we use the **MINRES-QLP** algoritm, which is specifically devised to solve least square problems of singular or nearly singular linear equations  [9,10,11,12].

In [ ]:
struct HodgeDecomposition
    f::Array{Real}
    s::Array{Real}
    h::Array{Real}
    g::Array{Real}
    u::Array{Real}
    w::Array{Real}    
    k::Int    
end

In [ ]:
function compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    if k<kappa(sc)
        alpha = sc.alpha[k+1]
        y = alpha*f
        L = alpha*alpha'
        u = Krylov.minres_qlp(L,y)[1]
        #u = Krylov.lsqr(L,y)[1]
        s = alpha'*u
        return s,u
    else
        s = zeros(m_k(sc,k)) # s = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        u = zeros(m_k(sc,k+1))        
        return s,u
    end
end

In [ ]:
function compute_k_th_gradient_hodge_decomposition(sc,f,k)
    if k>=1
        alpha = sc.alpha[k]
        z = alpha'*f
        L = alpha'*alpha
        w = Krylov.minres_qlp(L,z)[1]
        #w = Krylov.lsqr(L,z)[1]
        g = alpha*w
        return g,w
    else
        g = zeros(m_k(sc,k)) # g = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        w = zeros(m_k(sc,k-1))        
        return g,w
    end
end

In [ ]:
function compute_k_th_hodge_decomposition(sc,f,k)
    @assert length(f) == m_k(sc,k) @show (length(f),m_k(sc,k))
    s,u = compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    g,w = compute_k_th_gradient_hodge_decomposition(sc,f,k)
    h=f-s-g
    return HodgeDecomposition(f,s,h,g,u,w,k)
end

### Test

#### Basic test

Next, we create a 1-chain $f$ of components $f_{ij}=w_i-w_j$ where $w$ is the 0-chain of components $w_i=i$.

#### Test with futbol networks

# Ranking tools

Tools to generate **true** ratings and rankings for a given network.

In [ ]:
"""
Takes a vector and shifts the values of its components by a constant to make all of them non-negative.
"""
function ratings(w)
    return w .- minimum(w)
end

In [ ]:
"""
Normalizes a vector of ratings to have values between 0 and 1 inclusive.
"""
function normalized_ratings(w)
    minw = minimum(w)
    #maxw = maximum(w)
    #return (w .- minw)/(maxw-minw)
    return (w .- minw)/length(w)
end

In [ ]:
"""
Takes a vector of ratings and computes the corresponding ranking
"""
function ranking(w)
    return invperm(sortperm(w,rev=true))
end

In [ ]:
"""
Given two vectors v and u, this computes the absolute differences

    x_i = |v_i-u_i|

And returns the vector w of cumulative diferences

    w_i = sum_{j <= i} x_j

This can be used to compare two vectors of ratings or rankings
"""
function compute_cumulative_abs_diff(v,u)
    n = length(v)
    @assert length(u) == n
    cumulative_abs_diff = zeros(n)
    sum = 0
    for i in 1:n
         sum += abs.(v[i]-u[i])
         cumulative_abs_diff[i] = sum
    end
    return cumulative_abs_diff
end

### Test

## Load dict of weighted g's

In [ ]:
jld2_file_dict_weighted_g = "25-11-28-dict-weighted-g.jld2"
@load "jld2/"*jld2_file_dict_weighted_g dict_weighted_g_jld2

In [ ]:
jld2_file_true_ratings = "25-11-28-dict-true-ratings.jld2"
@load "jld2/"*jld2_file_true_ratings dict_true_ratings_jld2

### Test

## Weighted $g$ to ($K$,$f$)

In [ ]:
function g_to_sc_f(g)
    I_3c = g_to_3_cliques(g)
    sc = I_to_SC(I_3c)
    #m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    f = zeros(m_1)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = get_weight(g,i,j)
    end
    return (sc,f)
end

## Inferere of ratings and rankings

### Test with points

### Test with ranks

### Test with shots

## Systematic computation of ratings and rankings for Maximal Spanning Trees

### Sparsed list of weighted edges

In [ ]:
function get_network_wedges(g;sorted=true,rev=true,shuffled=false,fraction=1.0)
    wedges = collect(edges(g))
    if sorted
        sort!(wedges,lt=(x,y)->abs(get_weight(g,x))<abs(get_weight(g,y)),rev=rev)
    end
    if shuffled
        shuffle!(wedges)
    end
    #@show length(wedges)
    return wedges[1:round(Int64,length(wedges)*fraction)]
end

#### Test

In [ ]:
league = "Italy"
metric = "pressure_loss"
g = dict_weighted_g_jld2[(league,metric)]

In [ ]:
wedges = get_network_wedges(g,sorted=true,rev=true,shuffled=false,fraction=1.0)

### Maximal Spanning Tree

In [ ]:
function maximal_spanning_tree(g;sorted=true,rev=true,shuffled=false,loops=false,fraction=1.0)
    dus = DUS(nv(g))
    mst = SimpleWeightedGraph(nv(g)) # sparse graph
    wedges = get_network_wedges(g,sorted=sorted,rev=rev,shuffled=shuffled,fraction=fraction)
    for e in wedges
        s,t=src(e),dst(e)
        if find(dus,s) != find(dus,t) || loops
            #@show s,t
            add_edge!(mst,s,t,get_weight(g,e))
        end
        unite(dus,s,t)
        if dus.nds == 1
            break
        end
    end
    return mst
end

#### Test with MST

In [ ]:
mst = maximal_spanning_tree(g,shuffled=false,loops=true)

In [ ]:
gplot(mst)

In [ ]:
g_sc,g_f = g_to_sc_f(g)
g_hd = compute_k_th_hodge_decomposition(g_sc,g_f,1)
g_w = ratings(g_hd.w)
g_r = ranking(g_hd.w)

In [ ]:
mst_sc,mst_f = g_to_sc_f(mst)
mst_hd = compute_k_th_hodge_decomposition(mst_sc,mst_f,1)
mst_w = ratings(mst_hd.w)
mst_r = ranking(mst_hd.w)

In [ ]:
p = plot(
    xlabel="true ranking",
    ylabel="rating ["*metric*"]",
    title=league*" "*metric,
)
plot!(g_w,c=1,label="")
scatter!(g_w,c=1,label="g")
plot!(mst_w,c=1,label="",s=:dash)
scatter!(mst_w,c=1,label="mst",m=:star)
fname = "plots/tmp/"*league*"-"*metric*"-rating-mst.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="true ranking",
    ylabel="inferred ranking ["*metric*"]",
)
plot!(g_r,c=2,label="")
scatter!(g_r,c=2,label="g")
plot!(mst_r,c=2,label="",s=:dash)
scatter!(mst_r,c=2,label="mst",m=:star)
plot!(x->x,c=:black,style=:dash,label="ideal")
fname = "plots/tmp/"*league*"-"*metric*"-ranking-mst.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="g rating",
    ylabel="mst rating ["*metric*"]",
    title=league*" "*metric,
)
#plot!(g_w,mst_w,c=1,label="")
scatter!(g_w,mst_w,c=1,label="")
fname = "plots/tmp/"*league*"-"*metric*"-g-rating-vs-mst-rating.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="g ranking",
    ylabel="mst ranking ["*metric*"]",
    title=league*" "*metric,   
)
#plot!(g_r,mst_r,c=1,label="")
scatter!(g_r,mst_r,c=2,label="")
#plot!(mst_r,c=2,label="mst")
#scatter!(mst_r,c=2,label="")
plot!(x->x,c=:black,style=:dash,label="ideal")
fname = "plots/tmp/"*league*"-"*metric*"-g-ranking-vs-mst-ranking.pdf"
#@show fname
savefig(p,fname)
display(p)

#### Test with Shuffled Spanning Tree

In [ ]:
sst = maximal_spanning_tree(g,shuffled=true,loops=true)

In [ ]:
collect(edges(sst))

In [ ]:
gplot(sst)

In [ ]:
sst_sc,sst_f = g_to_sc_f(sst)
sst_hd = compute_k_th_hodge_decomposition(sst_sc,sst_f,1)
sst_w = ratings(sst_hd.w)
sst_r = ranking(sst_hd.w)

In [ ]:
p = plot(
    xlabel="true ranking",
    ylabel="rating ["*metric*"]",
    title=league*" "*metric,
)
plot!(g_w,c=1,label="")
scatter!(g_w,c=1,label="g")
plot!(sst_w,c=1,label="",s=:dash)
scatter!(sst_w,c=1,label="sst",m=:star)
fname = "plots/tmp/"*league*"-"*metric*"-rating-sst.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="true ranking",
    ylabel="inferred ranking ["*metric*"]",
)
plot!(g_r,c=2,label="")
scatter!(g_r,c=2,label="g")
plot!(sst_r,c=2,label="",s=:dash)
scatter!(sst_r,c=2,label="sst",m=:star)
plot!(x->x,c=:black,style=:dash,label="ideal")
fname = "plots/tmp/"*league*"-"*metric*"-ranking-sst.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="g rating",
    ylabel="sst rating ["*metric*"]",
    title=league*" "*metric,
)
#plot!(g_w,mst_w,c=1,label="")
scatter!(g_w,sst_w,c=1,label="")
fname = "plots/tmp/"*league*"-"*metric*"-g-rating-vs-sst-rating.pdf"
#@show fname
savefig(p,fname)
display(p)

In [ ]:
p = plot(
    xlabel="g ranking",
    ylabel="sst ranking ["*metric*"]",
    title=league*" "*metric,   
)
#plot!(g_r,sst_r,c=1,label="")
scatter!(g_r,sst_r,c=2,label="")
#plot!(mst_r,c=2,label="mst")
#scatter!(mst_r,c=2,label="")
plot!(x->x,c=:black,style=:dash,label="ideal")
fname = "plots/tmp/"*league*"-"*metric*"-g-ranking-vs-sst-ranking.pdf"
#@show fname
savefig(p,fname)
display(p)

### Systematic computation of rankings and ratings

In [ ]:
function save_ratings_and_rankings_as_txt(fname,w,r,tr;mode="a")
    fhw = open(fname,mode)
    if !isfile(fname)
        println(fhw,"i w_i r_i tr_i")
    end
    for i in 1:length(w)
        println(fhw,i," ",w[i]," ",r[i]," ",tr[i])
    end
    close(fhw)
end

#### Test

#### One sample for each (test)

In [ ]:
for (k,g) in dict_weighted_g_jld2
    @show k,g
    (league,metric) = k

    true_ratings = dict_true_ratings_jld2[league]
    
    #@show league,metric
    g_sc,g_f = g_to_sc_f(g)
    g_hd = compute_k_th_hodge_decomposition(g_sc,g_f,1)
    g_w = ratings(g_hd.w)
    g_r = ranking(g_hd.w)
    #@show k,w,r
    sst = maximal_spanning_tree(g,loops=true,shuffled=true)
    #sst = maximal_spanning_tree(g,loops=false,shuffled=true)
    sst_sc,sst_f = g_to_sc_f(sst)
    sst_hd = compute_k_th_hodge_decomposition(sst_sc,sst_f,1)
    sst_w = ratings(sst_hd.w)
    sst_r = ranking(sst_hd.w)

    #@show nv(g),ne(g),nv(sst),ne(sst)
    
    # Plot inferred rating
    p = plot(
        xlabel="true ranking",
        ylabel="rating ["*metric*"]",
    )
    plot!(g_w,c=1,label="")
    scatter!(g_w,c=1,label="g")
    plot!(sst_w,c=1,label="",style=:dash)
    scatter!(sst_w,c=1,label="sst",m=:star)
    fname = "plots/tmp/"*league*"-"*metric*"-rating.pdf"
    #@show fname
    savefig(p,fname)
    fname = "txt/tmp/"*league*"-"*metric*"-rating.txt"    
    #display(p)

    # Plot inferred ranking
    p = plot(
        xlabel="true ranking",
        ylabel="inferred ranking ["*metric*"]",
    )
    plot!(g_r,c=2,label="")
    scatter!(g_r,c=2,label="g")
    plot!(sst_r,c=2,label="",style=:dash)
    scatter!(sst_r,c=2,label="sst",m=:star)
    plot!(x->x,c=:black,style=:dash,label="ideal")
    fname = "plots/tmp/"*league*"-"*metric*"-ranking.pdf"
    #@show fname
    savefig(p,fname)
    #display(p)

    p = plot(
        xlabel="true rating",
        ylabel="sst rating",
        title=league*" "*metric,
    )
    scatter!(true_ratings,sst_w,c=1,label="")
    fname = "plots/tmp/"*league*"-"*metric*"-true-rating-vs-sst-rating.pdf"
    #@show fname
    savefig(p,fname)
    #display(p)
    
    p = plot(
        xlabel="g rating",
        ylabel="sst rating",
        title=league*" "*metric,
    )
    scatter!(g_w,sst_w,c=1,label="")
    fname = "plots/tmp/"*league*"-"*metric*"-g-rating-vs-sst-rating.pdf"
    #@show fname
    savefig(p,fname)
    #display(p)
    
    p = plot(
        xlabel="g ranking",
        ylabel="sst ranking",
        title=league*" "*metric,
    )
    scatter!(g_r,sst_r,c=2,label="")
    plot!(x->x,c=:black,style=:dash,label="ideal")    
    fname = "plots/tmp/"*league*"-"*metric*"-g-ranking-vs-sst-ranking.pdf"
    #@show fname
    savefig(p,fname)
    #display(p)

    #@show string.(collect(1:nv(sst)))
    p = gplot(sst,title=league*" "*metric,nodelabel = string.(collect(1:nv(sst))))
    fname = "plots/tmp/"*league*"-"*metric*"-sst.pdf"    
    #savefig(p,fname)
    draw(PDF(fname,16cm,16cm),p)

    # Save ratings and rankings as txt
    g_fname   = "txt/tmp/"*league*"-"*metric*"-g.txt"    
    save_ratings_and_rankings_as_txt(g_fname,g_w,g_r,true_ratings)        
    mst_fname = "txt/tmp/"*league*"-"*metric*"-sst.txt"    
    save_ratings_and_rankings_as_txt(mst_fname,sst_w,sst_r,true_ratings)            
end

#### Sampling sst's

In [ ]:
function save_histogram_as_txt(fname,h)
    open(fname,"w") do io
        ticks = collect(h.edges[1])
        print(io, "#ticks ")
        writedlm(io,ticks')
        print(io, "#weights ")
        writedlm(io,h.weights')
    end
end

In [ ]:
num_samples = 100

for (k,g) in dict_weighted_g_jld2
    @show k,g
    (league,metric) = k

    true_rankings = collect(1:nv(g))    
    true_ratings  = dict_true_ratings_jld2[league]
    
    g_sc,g_f = g_to_sc_f(g)
    g_hd = compute_k_th_hodge_decomposition(g_sc,g_f,1)
    g_w = ratings(g_hd.w)
    g_r = ranking(g_hd.w)

    mst = maximal_spanning_tree(g,loops=true)
    mst_sc,mst_f = g_to_sc_f(mst)
    mst_hd = compute_k_th_hodge_decomposition(mst_sc,mst_f,1)
    mst_w = ratings(mst_hd.w)
    mst_r = ranking(mst_hd.w)

    list_inferred_sst_ratings = zeros(0)
    list_inferred_sst_rankings = zeros(Int64,0)

    list_rs_sst_ratings  = [RunningStat() for i in 1:nv(g)]
    list_rs_sst_rankings = [RunningStat() for i in 1:nv(g)]
    
    # Sampling sst's
    for sample in 1:num_samples
        if sample%10 == 0
            @show sample
        end
        sst = maximal_spanning_tree(g,loops=true,shuffled=true)
        sst_sc,sst_f = g_to_sc_f(sst)
        sst_hd = compute_k_th_hodge_decomposition(sst_sc,sst_f,1)
        sst_w = ratings(sst_hd.w)
        sst_r = ranking(sst_hd.w)      
        
        #push!(list_inferred_sst_ratings,sst_w)
        append!(list_inferred_sst_ratings,sst_w)
        #push!(list_inferred_sst_rankings,sst_r)
        append!(list_inferred_sst_rankings,sst_r)

        for i in 1:nv(g)
            push!(list_rs_sst_ratings[i],sst_w[i])
            push!(list_rs_sst_rankings[i],sst_r[i])
        end
        
        sst_fname = "txt/"*league*"-"*metric*"-sst.txt"    
        save_ratings_and_rankings_as_txt(sst_fname,sst_w,sst_r,true_ratings)        
    end
   
    p1 = plot(
        xlabel="true ranking",
        ylabel="rating ["*metric*"]",
    )
    #
    scatter!(p1,repeat(true_rankings;outer=[num_samples]),list_inferred_sst_ratings,c=3,label="",marker = (:circle, 3),alpha = 0.1,markerstrokewidth = 0)
    #
    plot!(p1,true_rankings,g_w,c=1,label="")
    scatter!(p1,true_rankings,g_w,c=1,label="g inferred",markerstrokewidth = 0)
    #
    plot!(p1,true_rankings,mst_w,c=2,label="")
    scatter!(p1,true_rankings,mst_w,c=2,markerstrokecolor=2,label="mst",markerstrokewidth = 0)
    #
    plot!(p1,true_rankings,mean.(list_rs_sst_ratings),c=3,label="")
    scatter!(p1,true_rankings,mean.(list_rs_sst_ratings),yerror=std.(list_rs_sst_ratings),c=3,markerstrokecolor=3,label="sst",markerstrokewidth = 0)
    #
    #plot!(p1,x->x,c=:black,style=:dash,label="ideal")        
    fname = "plots/"*league*"-"*metric*"-rating-samples.pdf"
    savefig(p1,fname)
    
    p2 = plot(
        xlabel="true ranking",
        ylabel="ranking ["*metric*"]",
    )   
    #
    scatter!(p2,repeat(true_rankings;outer=[num_samples]),list_inferred_sst_rankings,c=3,label="",marker = (:circle, 3),alpha = 0.1,markerstrokewidth = 0)
    #
    plot!(p2,true_rankings,g_r,c=1,label="")
    scatter!(p2,true_rankings,g_r,c=1,label="g inferred",markerstrokewidth = 0)
    #
    plot!(p2,true_rankings,mst_r,c=2,label="")
    scatter!(p2,true_rankings,mst_r,c=2,label="mst",markerstrokewidth = 0)
    #
    plot!(p2,true_rankings,mean.(list_rs_sst_rankings),c=3,label="")    
    scatter!(p2,true_rankings,mean.(list_rs_sst_rankings),yerror=std.(list_rs_sst_rankings),c=3,markerstrokecolor=3,label="sst",markerstrokewidth = 0)
    #
    plot!(p2,x->x,c=:black,style=:dash,label="ideal")        
    fname = "plots/"*league*"-"*metric*"-ranking-samples.pdf"
    savefig(p2,fname)

    p3 = plot(
        xlabel="true rating",
        ylabel="rating",
        title=league*" "*metric,
    )
    #
    scatter!(p3,repeat(true_ratings;outer=[num_samples]),list_inferred_sst_ratings,c=3,label="",marker = (:circle, 3),alpha = 0.1,markerstrokewidth = 0)
    #
    scatter!(p3,true_ratings,g_w,c=1,label="g inferred",markerstrokewidth = 0)     
    #
    scatter!(p3,true_ratings,mst_w,c=2,label="mst",markerstrokewidth = 0)         
    #
    scatter!(p3,true_ratings,mean.(list_rs_sst_ratings),yerror=std.(list_rs_sst_ratings),c=3,markerstrokecolor=3,label="sst",markerstrokewidth = 0)
    #
    #plot!(p3,x->x,c=:black,style=:dash,label="ideal")        
    fname = "plots/"*league*"-"*metric*"-true-rating-vs-sst-rating-samples.pdf"
    savefig(p3,fname)
    
    p4 = plot(
        xlabel="g rating",
        ylabel="rating",
        title=league*" "*metric,
    )
    #
    scatter!(p4,repeat(g_w;outer=[num_samples]),list_inferred_sst_ratings,c=3,label="",marker = (:circle, 3),alpha = 0.1,markerstrokewidth = 0)
    #
    scatter!(p4,g_w,mst_w,c=2,label="mst",markerstrokewidth = 0)         
    #
    scatter!(p4,g_w,mean.(list_rs_sst_ratings),yerror=std.(list_rs_sst_ratings),c=3,markerstrokecolor=3,label="sst",markerstrokewidth = 0)
    #
    plot!(p4,x->x,c=:black,style=:dash,label="ideal")
    fname = "plots/"*league*"-"*metric*"-g-rating-vs-sst-rating-samples.pdf"
    savefig(p4,fname)
    
    p5 = plot(
        xlabel="g ranking",
        ylabel="ranking",
        title=league*" "*metric,
    )
    #
    scatter!(p5,repeat(g_r;outer=[num_samples]),list_inferred_sst_rankings,c=3,label="",marker = (:circle, 3),alpha = 0.1,markerstrokewidth = 0)
    #
    scatter!(p5,g_r,mst_r,c=2,label="mst",markerstrokewidth = 0)
    #
    scatter!(p5,g_r,mean.(list_rs_sst_rankings),yerror=std.(list_rs_sst_rankings),c=3,markerstrokecolor=3,label="sst",markerstrokewidth = 0)
    #
    plot!(p5,x->x,c=:black,style=:dash,label="ideal")        
    fname = "plots/"*league*"-"*metric*"-g-ranking-vs-sst-ranking-samples.pdf"
    savefig(p5,fname)   

    # distributions in Julia
    # https://chatgpt.com/share/69179b8f-cbd0-8001-a8c6-746db44cb68b
    p6 = plot(
        xlabel="sst rating",
        ylabel="P(sst rating)",
        title=league*" "*metric,
    )
    #@show sst_rating_vs_g_rating.yvals
    hist = StatsBase.fit(Histogram,list_inferred_sst_ratings,nbins=50)
    probs = hist.weights ./ sum(hist.weights)
    plot!(p6,hist,label="")
    #plot!(p6,probs,label="")    
    fname = "plots/"*league*"-"*metric*"-distribution-sst-rating-samples.pdf"
    savefig(p6,fname)  
    fname = "txt/"*league*"-"*metric*"-distribution-sst-rating-samples.txt"
    save_histogram_as_txt(fname,hist)

    #@show hist
    
end